# Lab 2 - OpenAI Agents SDK!

2 steps to making an Agent:

1. Create a new class:

`agent = Agent(...)`

2. Call Runner.run

`Runner.run(agent, input)`

For this first part we will explore:

- The System Prompt with instructions
- Runner.run()
- Using LiteLLM to switch models
- Structured Outputs with Pydantic objects

In [ ]:
from agents import Agent, Runner
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
import os
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv
load_dotenv(override=True)


In [ ]:
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-4.1-mini")

In [ ]:
input = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:

Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.

Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.

If both steal, you both get nothing.

Do you choose to Steal or Share? Pick one.
"""

In [ ]:
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

## Now let's use LiteLLM to switch up to different models

Here are all the providers:

https://docs.litellm.ai/docs/providers

In [ ]:
model = LitellmModel(model="xai/grok-4", api_key=os.getenv("GROK_API_KEY"))
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

In [ ]:
model = LitellmModel(model="deepseek/deepseek-reasoner")
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

In [ ]:
# The famous trolley dilemma

input = """
A runaway trolley is heading down a track. Ahead, five people are tied to the tracks and will be killed if the trolley continues.

You are standing next to a lever. If you pull it, the trolley will switch to a different track — but one person is tied to that one.

Do you pull the lever? Choose to pull or not to pull.
"""

## Structured Outputs

In the next cell, we define a Pydantic object.

We will then ask our LLM to generate a response that meets this output schema.

In [ ]:
class Decision(BaseModel):
    reasoning: str = Field(description="The rationale for your decision")
    counter_argument: str = Field(description="A counter-argument to the reasoning")
    pull_lever: bool = Field(description="Whether to pull the lever")

In [ ]:
autonomous_agent_with_structure = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-5-nano", output_type=Decision)
result = await Runner.run(autonomous_agent_with_structure, input)
decision = result.final_output_as(Decision)
print("Pull lever?", decision.pull_lever)
print("Reasoning:", decision.reasoning)
print("Counter-argument:", decision.counter_argument)
